In [1]:
import os, time, json, csv
from datetime import datetime, timedelta
import logging

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
def log_setup(name, log_file):
    """Setup log for the whole process
    Parameters:
    `log_file` (string): Log file name.
    """
    log_handler = logging.handlers.TimedRotatingFileHandler(
    log_file, when="midnight", backupCount=3)
    formatter = logging.Formatter(
    "%(asctime)s - %(levelname)s - [%(process)d]: %(message)s",
    "%Y-%m-%d %H:%M:%S")
    log_handler.setFormatter(formatter)
    log_obj = logging.getLogger(name)
    log_obj.addHandler(log_handler)
    log_obj.setLevel(logging.INFO)
    
    return log_obj

logFile = log_setup('logFile', "log/crawling-gmaps.log")

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")
# chrome_options.add_argument("--headless")

In [4]:
chrome_install = ChromeDriverManager().install()

folder = os.path.dirname(chrome_install)
chromedriver_path = os.path.join(folder, "chromedriver.exe")

desired_capabilities = DesiredCapabilities.CHROME
driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)
driver.maximize_window()

C:\Users\dendy\AppData\Local\Temp/ipykernel_67256/4267024452.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)


In [6]:
driver.get("https://www.google.com/maps/?hl=en")

In [8]:
# find_poi = 'atm'
find_poi = 'atm bni'
f_city_ind = 0
l_city_ind = 10

In [9]:
with open('list_city.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
list_city = [data[idx][0] if (idx >= f_city_ind) and (idx <= l_city_ind) else None for idx in range(len(data))]
list_city = [city for city in list_city if city is not None]
list_city

['KABUPATEN ACEH BARAT ACEH',
 'KABUPATEN ACEH BARAT DAYA ACEH',
 'KABUPATEN ACEH BESAR ACEH',
 'KABUPATEN ACEH JAYA ACEH',
 'KABUPATEN ACEH SELATAN ACEH',
 'KABUPATEN ACEH SINGKIL ACEH',
 'KABUPATEN ACEH TAMIANG ACEH',
 'KABUPATEN ACEH TENGAH ACEH',
 'KABUPATEN ACEH TENGGARA ACEH',
 'KABUPATEN ACEH TIMUR ACEH',
 'KABUPATEN ACEH UTARA ACEH']

In [10]:
logFile.info(f'Start Crawling for POI: {find_poi}')
resultFile = log_setup('resultFile', f'result/data_gmaps_poi_{find_poi.replace(" ","_")}_{f_city_ind}_{l_city_ind}_{datetime.now().strftime("%Y%m")}.txt')

In [12]:
file_done_city = f'done/list_city_done_{find_poi.replace(" ","_")}_{f_city_ind}_{l_city_ind}_{datetime.now().strftime("%Y%m")}.csv'
if os.path.exists(file_done_city):
    with open(file_done_city, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
    list_done_city = [x[0] for x in data]
else:
    list_done_city = []
list_done_city

[]

In [20]:
city = "KABUPATEN BEKASI JAWA BARAT"
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').clear()
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(f"{city}")

In [23]:
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(Keys.RETURN)

In [22]:
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').clear()
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(f"{find_poi} in {city}")

In [24]:
els = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
els2 = els.find_elements(By.CLASS_NAME, 'hfpxzc')
last_el_result = len(els2)
try_cnt = 5
while (1):
    result_el = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    scrollTop = driver.execute_script("return arguments[0].scrollTop", result_el)
    scrollHeight = driver.execute_script("return arguments[0].scrollHeight", result_el)
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight + 5000", result_el)
    els = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    els2 = els.find_elements(By.CLASS_NAME, 'hfpxzc')
    if last_el_result == len(els2):
        if try_cnt < 0:
            break
        else:
            try_cnt -= 1
    else:
        last_el_result = len(els2)
        try_cnt = 5
    time.sleep(1)
print(f"Total Data: {last_el_result}")
logFile.info(f"Total Data: {last_el_result}")

Total Data: 120


In [25]:
idx = 1
for el in els2:
    data = {}
    data['city'] = city
    data['name'] = el.get_attribute('aria-label')
    parent_el = el.find_element(By.XPATH, '..')
    # //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[3]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span
    data['cat'] = parent_el.find_element(By.XPATH, './div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span').text
    data['link'] = el.get_attribute('href')
    # print(data['link'])
    dt_href = data['link'].split("/")[6].split("=")[1].split("!")
    data['lat'] = list(filter(lambda item: item is not None, [x[2:] if x[:2].find("3d") > -1 else None for x in dt_href]))[0]
    data['lon'] = list(filter(lambda item: item is not None, [x[2:] if x[:2].find("4d") > -1 else None for x in dt_href]))[0].split('?')[0]
    data["crawling_date"] = datetime.now().strftime("%d-%m-%Y %H:%M:%S")

    # print(f"{idx}. {data['name']} - {data['cat']} - {data['lat']} - {data['lon']} - {data['link']}")
    logFile.info(f"{idx}. {data['name']} - {data['cat']} - {data['lat']} - {data['lon']} - {data['link']}")
    resultFile.info(f"result##{json.dumps(data)}")
    idx+=1

In [31]:
test_data = 'https://www.google.com/maps/place/ATM+BNI/data=!4m7!3m6!1s0x2e6999e5ce9808e9:0xe76076ac3991649a!8m2!3d-6.385615!4d107.1104503!16s%2Fg%2F11clht0n63!19sChIJ6QiYzuWZaS4RmmSROax2YOc?authuser=0&hl=en&rclk=1'
test_data.split('/')[6].split('=')[1].split('!')

['',
 '4m7',
 '3m6',
 '1s0x2e6999e5ce9808e9:0xe76076ac3991649a',
 '8m2',
 '3d-6.385615',
 '4d107.1104503',
 '16s%2Fg%2F11clht0n63',
 '19sChIJ6QiYzuWZaS4RmmSROax2YOc?authuser']

In [ ]:
for city in list_city:
    if city != None and city not in list_done_city:
        # city = 'KABUPATEN ACEH JAYA ACEH'
        logFile.info(f'Start Crawling for City: {city}')
        driver.get("https://www.google.com/maps/?hl=en")
        time.sleep(2)
        # //*[@id="searchboxinput"]
        # /html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[1]/form/input
        driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(f"{city}")
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(Keys.RETURN)
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').clear()
        driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(f"{find_poi} in {city}")
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(Keys.RETURN)
        time.sleep(2)

        # //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]
        els = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
        els2 = els.find_elements(By.CLASS_NAME, 'hfpxzc')
        if len(els2) > 2:
            last_el_result = len(els2)
            try_cnt = 5
            while (1):
                result_el = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
                scrollTop = driver.execute_script("return arguments[0].scrollTop", result_el)
                scrollHeight = driver.execute_script("return arguments[0].scrollHeight", result_el)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight + 5000", result_el)
                els = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
                els2 = els.find_elements(By.CLASS_NAME, 'hfpxzc')
                if last_el_result == len(els2):
                    if try_cnt < 0:
                        break
                    else:
                        try_cnt -= 1
                else:
                    last_el_result = len(els2)
                    try_cnt = 5
                time.sleep(1)
            print(f"Total Data: {last_el_result}")
            logFile.info(f"Total Data: {last_el_result}")

            idx = 1
            for el in els2:
                data = {}
                data['city'] = city
                data['name'] = el.get_attribute('aria-label')
                parent_el = el.find_element(By.XPATH, '..')
                data['cat'] = parent_el.find_element(By.XPATH, './div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span').text
                data['link'] = el.get_attribute('href')
                # print(data['link'])
                dt_href = data['link'].split("/")[6].split("=")[1].split("!")
                data['lat'] = list(filter(lambda item: item is not None, [x[2:] if x[:2].find("3d") > -1 else None for x in dt_href]))[0]
                data['lon'] = list(filter(lambda item: item is not None, [x[2:] if x[:2].find("4d") > -1 else None for x in dt_href]))[0].split('?')[0]
                data["crawling_date"] = datetime.now().strftime("%d-%m-%Y %H:%M:%S")

                # print(f"{idx}. {data['name']} - {data['cat']} - {data['lat']} - {data['lon']} - {data['link']}")
                logFile.info(f"{idx}. {data['name']} - {data['cat']} - {data['lat']} - {data['lon']} - {data['link']}")
                resultFile.info(f"result##{json.dumps(data)}")
                idx+=1
        
        list_done_city.append(city)
        with open(file_done_city, 'w+') as f:
            f.write("\n".join(list_done_city))